**SME0213 Otimização Inteira - ICMC-USP**

Amanda Araujo Silva - 10260441

In [6]:
%pip install gurobipy

In [2]:
import gurobipy as gp

# Atividade Complementar 2: Corte de jumbo



## Problema de corte

Problema clássico de otimização: problema de corte.

$\quad$ Dados do problema:

*    Comprimento bobina jumbo $L$
*    Comprimentos dos itens $l_i$: comprimento item $i$
*    Demanda de cada item $d_i$: demanda item $i$

$\quad$Modelagem geral (Kantorovich):

> $Min \sum_{k = 1}^K y_k$

> $S.a. \sum_{k = 1}^K \alpha_{jk} \ge d_j \quad j = 1, ..., n$

> $l_1\alpha_{1k} + l_2\alpha_{2k} + ... + l_n\alpha_{nk} \le L y_k \quad k = 1, ..., K$

> $\alpha_{jk} \ge 0$ e inteiro $\quad j = 1, ..., n \quad k = 1, ..., K$

> $y_k \in \{0, 1\} \quad k = 1, ..., K$

sendo:
*    $y_k$: variável de decisão binária; 1 se a barra $k$ é cortada, 0 c.c.;
*    $\alpha_{jk}$: variável de decisão inteira; número de vezes que o item $j$ é cortado na barra $k$;
*    $K$: valor limite para o número de barras (estimativa);
*    $n$: número de itens.

## Implementação Kantorovich

In [13]:
from gurobipy import Model, GRB
import math

# Criação do Modelo
model = Model("jumbo")

# Dados do problema
L = 400.0 # tamanho bobina jumbo
l = [40.0, 45.0, 55.0, 60.0] # tamanho itens
d = [12.0, 20.0, 42.0, 18.0] # demanda itens

## Parâmetros
n = 4   # número de itens
K = (int(math.ceil( sum(l[i] * d[i] for i in range(n))/L ))) # valor limite nº de barras; menor limiar
print("Tamanho bobina jumbo:", L)
print("Demanda total:", sum(l[i] * d[i] for i in range(n)))
print("Valor limite K: ", K)
print("\n")

# Adição de variáveis
alpha = model.addVars(n, K, vtype=GRB.INTEGER, name="alpha", lb=0) # inteira não negativa Z+
y = model.addVars(K, vtype=GRB.BINARY, name="y")                   # binária

# Definição função objetivo: minimizar número de barras utilizadas
model.setObjective(
    sum(y[k] for k in range(K)),
    GRB.MINIMIZE
)

# Restrições
## Demanda
for j in range(n):
  model.addConstr(sum(alpha[j, k] for k in range(K)) >= d[j])

## Tamanho bobina jumbo
for k in range(K):
  model.addConstr(sum(l[i] * alpha[i, k] for i in range(n)) <= L * y[k])

## Restrição alpha >= 0 já add pelo uso de lb (lower bound) ao definir alpha
## Restrição y binário já add ao definir y

# Otimização
model.optimize()


# Escrita do modelo em formato LP
model.write('jumbo.lp')

# Salva a solução do problema
model.write('jumbo.sol')

# Exibe os resultados
if model.status == GRB.OPTIMAL:
    for i in range(n):
      for k in range(K):
        if alpha[i, k].X > 0:
          print(f"alpha[{i}, {k}] = {alpha[i, k].X}")

    for k in range(K):
      if y[k].X > 0:
        print(f"y[{k}] = {y[k].X}")

    print(f"z = {model.ObjVal}")

else:
    print("Solução ótima não encontrada.")

Tamanho bobina jumbo: 400.0
Demanda total: 4770.0
Valor limite K:  12


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 60 columns and 108 nonzeros
Model fingerprint: 0x99dd66f5
Variable types: 0 continuous, 60 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 4e+01]
Presolve time: 0.00s
Presolved: 16 rows, 60 columns, 108 nonzeros
Variable types: 0 continuous, 60 integer (12 binary)

Root relaxation: objective 1.192500e+01, 45 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   11.92500    0    

## Padrões de corte encontrados, usos e perdas


Cada padrão de corte representa como uma bobina jumbo é utilizada, ou seja, quantas unidades de cada item são cortadas dela: representado pelas variáveis alpha[i, k].

*    $\alpha_{jk}$: número de vezes que o item $j$ é cortado na barra $k$

### Padrão de corte por bobina

In [4]:
# Padrões de corte encontrados (por barra)

print("\n" + "=" * 45)
print("Padrões de corte encontrados (por bobina):")
print("=" * 45)

## Cabeçalho com os comprimentos dos itens
header = "Bobina | " + " | ".join([f"{int(li):>4}" for li in l])
print(header)
print("-" * (len(header) + 3))

## Para cada bobina usada, mostrar a linha com os cortes
for k in range(K):
    if y[k].X > 0.5:
        linha = f"{k:>6} |"
        for i in range(n):
            qtd = int(alpha[i, k].X + 0.5)
            cell = f"{qtd:>4}" if qtd > 0 else "    "
            linha += f" {cell} |"
        print(linha)

print("=" * 45)
print(f"Total de bobinas utilizadas: {int(model.ObjVal)}")
print("=" * 45)


Padrões de corte encontrados (por bobina):
Bobina |   40 |   45 |   55 |   60
-------------------------------------
     0 |      |    5 |    1 |    2 |
     1 |    1 |    3 |    4 |      |
     2 |      |      |    5 |    2 |
     3 |      |      |    4 |    3 |
     4 |      |    4 |    4 |      |
     5 |    3 |      |    4 |    1 |
     6 |    1 |    3 |    3 |    1 |
     7 |      |      |    5 |    2 |
     8 |      |    1 |    2 |    4 |
     9 |    3 |      |    5 |      |
    10 |    3 |      |    5 |      |
    11 |    1 |    4 |      |    3 |
Total de bobinas utilizadas: 12


### Padrões de corte únicos, usos e perdas

In [5]:
# Padrões de corte únicos encontrados, usos e perdas
from collections import defaultdict

# Contar padrões de corte e calcular perdas
padroes_unicos = defaultdict(int)
padroes_perda = {}

for k in range(K):
    if y[k].X > 0.5:
        padrao = tuple(int(alpha[i, k].X + 0.5) for i in range(n))
        padroes_unicos[padrao] += 1
        if padrao not in padroes_perda:
            usado = sum(padrao[i] * l[i] for i in range(n))
            perda = L - usado
            padroes_perda[padrao] = perda

print(f"Total de bobinas utilizadas: {int(model.ObjVal)}")
print(f"\nNúmero de padrões de corte distintos: {len(padroes_unicos)}")

# Exibir a tabela final
print("\n" + "=" * 45)
print(f"{'Padrão de corte':<20} | {'Usos':^5} | {'Perda':^6}")
print("-" * 45)

for padrao in padroes_unicos:
    usos = padroes_unicos[padrao]
    perda = padroes_perda[padrao]
    print(f"{str(padrao):<20} | {usos:^5} | {perda:^6.1f}")

print("=" * 45)

# Perda total
perda_total = sum(padroes_unicos[p] * padroes_perda[p] for p in padroes_unicos)
print(f"Perda total acumulada: {perda_total:.1f}")
print("=" * 45)


Total de bobinas utilizadas: 12

Número de padrões de corte distintos: 10

Padrão de corte      | Usos  | Perda 
---------------------------------------------
(0, 5, 1, 2)         |   1   |  0.0  
(1, 3, 4, 0)         |   1   |  5.0  
(0, 0, 5, 2)         |   2   |  5.0  
(0, 0, 4, 3)         |   1   |  0.0  
(0, 4, 4, 0)         |   1   |  0.0  
(3, 0, 4, 1)         |   1   |  0.0  
(1, 3, 3, 1)         |   1   |  0.0  
(0, 1, 2, 4)         |   1   |  5.0  
(3, 0, 5, 0)         |   2   |  5.0  
(1, 4, 0, 3)         |   1   |  0.0  
Perda total acumulada: 30.0
